<a href="https://colab.research.google.com/github/andrewsongj/fraud-detector/blob/main/NetSec_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

In [ ]:
! pip install -q kaggle

from google.colab import files

In [ ]:
files.upload()

In [ ]:
! mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json



In [ ]:
!kaggle datasets download -d emilynack/aci-iot-network-traffic-dataset-2023

100% 32.3G/32.3G [20:55<00:00, 26.0MB/s]
100% 32.3G/32.3G [20:55<00:00, 27.6MB/s]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!mv aci-iot-network-traffic-dataset-2023.zip /content/drive/MyDrive/

mv: cannot stat 'aci-iot-network-traffic-dataset-2023.zip': No such file or directory


In [ ]:
%%capture
!mv aci-iot-network-traffic-dataset-2023.zip /content/drive/MyDrive/
import pandas as pd
!unzip /content/drive/MyDrive/aci-iot-network-traffic-dataset-2023.zip

In [ ]:
df = pd.read_csv("ACI-IoT-2023.csv")

In [ ]:
df.drop(["Flow Bytes/s","Timestamp","Flow Packets/s"],axis=1,inplace=True)

In [ ]:
selected_labels = ['ICMP Flood', 'Slowloris', 'SYN Flood', 'UDP Flood', 'DNS Flood', 'Benign']
df =df[df['Label'].isin(selected_labels)]

In [ ]:
features = df.columns.tolist()
features.remove("Label")
features.remove("Flow ID")
features.remove("Src IP")
features.remove("Dst IP")
X = df[features]
y=df['Label']

In [ ]:
X

,Src Port,Dst Port,Protocol,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Connection Type
0,33344,443,6,379933,11,11,720.0,6169.0,517.0,0.0,...,32,0.000000e+00,0.000000e+00,0.0,0.0,1.698846e+15,0.000000e+00,1.698846e+15,1.698846e+15,wired
1,36754,80,6,205637,3,3,87.0,189.0,87.0,0.0,...,32,0.000000e+00,0.000000e+00,0.0,0.0,1.698846e+15,0.000000e+00,1.698846e+15,1.698846e+15,wired
2,36754,80,6,0,2,0,0.0,0.0,0.0,0.0,...,32,0.000000e+00,0.000000e+00,0.0,0.0,1.698846e+15,0.000000e+00,1.698846e+15,1.698846e+15,wired
3,8080,40426,6,5030379,1,2,6.0,0.0,6.0,6.0,...,20,0.000000e+00,0.000000e+00,0.0,0.0,8.494231e+14,1.201266e+15,1.698846e+15,5.030379e+06,wired
4,40054,80,6,72278,3,4,87.0,148.0,87.0,0.0,...,32,0.000000e+00,0.000000e+00,0.0,0.0,1.698846e+15,0.000000e+00,1.698846e+15,1.698846e+15,wired
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1231406,58126,443,6,1,2,0,12.0,0.0,6.0,6.0,...,20,0.000000e+00,0.000000e+00,0.0,0.0,1.698946e+15,0.000000e+00,1.698946e+15,1.698946e+15,wired
1231407,30001,47579,6,50077229,4,2,140.0,390.0,70.0,0.0,...,32,7.708200e+04,0.000000e+00,77082.0,77082.0,8.494730e+14,1.201336e+15,1.698946e+15,4.992308e+07,wired
1231408,0,0,0,46365348,15,0,0.0,0.0,0.0,0.0,...,0,3.713166e+06,3.789721e+06,8045778.0,1014561.0,4.247365e+14,8.494730e+14,1.698946e+15,6.831749e+06,wired
1231409,52046,443,6,146262,15,37,20340.0,14242.0,9327.0,0.0,...,20,0.000000e+00,0.000000e+00,0.0,0.0,1.698946e+15,0.000000e+00,1.698946e+15,1.698946e+15,wired


In [ ]:
# prompt: one hot encode connection field in X

import pandas as pd
X = pd.get_dummies(X, columns=["Connection Type"],dtype=int)


In [ ]:
# prompt: prep data for keras

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0, stratify=y)
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
y_train_encoded = encoder.fit_transform(y_train)
y_test_encoded = encoder.transform(y_test)
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import BatchNormalization


# Build the model
model = Sequential([
    Dense(256, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dropout(0.3),
    Dense(128, activation='relu'),
    Dropout(0,3)

    Dense(64, activation='relu'),
    Dense(len(encoder.classes_), activation='softmax')  # Output layer
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_16 (Dense)            (None, 256)               20480     
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_17 (Dense)            (None, 128)               32896     
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_18 (Dense)            (None, 64)                8256      
                                                                 
 dense_19 (Dense)            (None, 6)                 390       
                                                                 
Total params: 62022 (242.27 KB)
Trainable params: 62022 

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam

model = Sequential([
        Dense(256, activation='relu', input_shape=(X_train_scaled.shape[1],)),
        BatchNormalization(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        BatchNormalization(),
        Dense(64, activation='relu'),
        Dense(len(encoder.classes_), activation='softmax')  # Output layer
    ])
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])



In [ ]:
history = model.fit(X_train_scaled, y_train_encoded, epochs=10, batch_size=32, validation_split=0.2)
test_loss, test_acc = model.evaluate(X_test_scaled, y_test_encoded)
print(f"Test Accuracy: {test_acc}")

Epoch 1/10
9522/9522 [==============================] - 47s 5ms/step - loss: 0.0497 - accuracy: 0.9848 - val_loss: 0.0714 - val_accuracy: 0.9740
Epoch 2/10
9522/9522 [==============================] - 45s 5ms/step - loss: 0.0223 - accuracy: 0.9941 - val_loss: 0.0159 - val_accuracy: 0.9959
Epoch 3/10
9522/9522 [==============================] - 46s 5ms/step - loss: 0.0189 - accuracy: 0.9951 - val_loss: 0.1137 - val_accuracy: 0.9708
Epoch 4/10
9522/9522 [==============================] - 46s 5ms/step - loss: 0.0198 - accuracy: 0.9946 - val_loss: 0.0116 - val_accuracy: 0.9967
Epoch 5/10
9522/9522 [==============================] - 46s 5ms/step - loss: 0.0164 - accuracy: 0.9955 - val_loss: 0.0172 - val_accuracy: 0.9963
Epoch 6/10
9522/9522 [==============================] - 46s 5ms/step - loss: 0.0157 - accuracy: 0.9958 - val_loss: 0.0110 - val_accuracy: 0.9971
Epoch 7/10
9522/9522 [==============================] - 45s 5ms/step - loss: 0.0148 - accuracy: 0.9961 - val_loss: 0.0103 - val_ac

In [ ]:
model.save('/content/drive/MyDrive/my_network_model.h5')  # Saves as a single HDF5 file

In [ ]:
from tensorflow.keras.models import load_model
base_model = tf.keras.models.load_model('/content/drive/MyDrive/my_network_model.h5')



In [ ]:
for layer in base_model.layers[:-1]:
    layer.trainable = False

# Print the trainable status of each layer
for layer in base_model.layers:
    print(layer, layer.trainable)

<keras.src.layers.core.dense.Dense object at 0x7856b47d6800> False
<keras.src.layers.regularization.dropout.Dropout object at 0x7856b47d5e10> False
<keras.src.layers.core.dense.Dense object at 0x7856b47d7d90> False
<keras.src.layers.regularization.dropout.Dropout object at 0x784f5b588910> False
<keras.src.layers.core.dense.Dense object at 0x784f5b588850> False
<keras.src.layers.core.dense.Dense object at 0x784f5b588040> True


In [ ]:
!kaggle datasets download -d cicdataset/cicids2017

100% 230M/230M [00:07<00:00, 40.6MB/s]
100% 230M/230M [00:07<00:00, 30.5MB/s]


In [ ]:
!mv cicids2017.zip /content/drive/MyDrive/

In [ ]:
!unzip /content/drive/MyDrive/cicids2017.zip

Archive:  /content/drive/MyDrive/cicids2017.zip
  inflating: MachineLearningCSV.md5  
  inflating: MachineLearningCSV/MachineLearningCVE/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv  
  inflating: MachineLearningCSV/MachineLearningCVE/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv  
  inflating: MachineLearningCSV/MachineLearningCVE/Friday-WorkingHours-Morning.pcap_ISCX.csv  
  inflating: MachineLearningCSV/MachineLearningCVE/Monday-WorkingHours.pcap_ISCX.csv  
  inflating: MachineLearningCSV/MachineLearningCVE/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv  
  inflating: MachineLearningCSV/MachineLearningCVE/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv  
  inflating: MachineLearningCSV/MachineLearningCVE/Tuesday-WorkingHours.pcap_ISCX.csv  
  inflating: MachineLearningCSV/MachineLearningCVE/Wednesday-workingHours.pcap_ISCX.csv  


In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
nRowsRead = None

df1 = pd.read_csv("MachineLearningCSV/MachineLearningCVE/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv")
df2=pd.read_csv("MachineLearningCSV/MachineLearningCVE/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv")
df3=pd.read_csv("MachineLearningCSV/MachineLearningCVE/Friday-WorkingHours-Morning.pcap_ISCX.csv")
df4=pd.read_csv("MachineLearningCSV/MachineLearningCVE/Monday-WorkingHours.pcap_ISCX.csv")
df5=pd.read_csv("MachineLearningCSV/MachineLearningCVE/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv")
df6=pd.read_csv("MachineLearningCSV/MachineLearningCVE/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv")
df7=pd.read_csv("MachineLearningCSV/MachineLearningCVE/Tuesday-WorkingHours.pcap_ISCX.csv")
df8=pd.read_csv("MachineLearningCSV/MachineLearningCVE/Wednesday-workingHours.pcap_ISCX.csv")


df = pd.concat([df1,df2])
del df1,df2
df = pd.concat([df,df3])
del df3
df = pd.concat([df,df4])
del df4
df = pd.concat([df,df5])
del df5
df = pd.concat([df,df6])
del df6
df = pd.concat([df,df7])
del df7
df = pd.concat([df,df8])
del df8

In [ ]:
df.columns

Index([' Destination Port', ' Flow Duration', ' Total Fwd Packets',
       ' Total Backward Packets', 'Total Length of Fwd Packets',
       ' Total Length of Bwd Packets', ' Fwd Packet Length Max',
       ' Fwd Packet Length Min', ' Fwd Packet Length Mean',
       ' Fwd Packet Length Std', 'Bwd Packet Length Max',
       ' Bwd Packet Length Min', ' Bwd Packet Length Mean',
       ' Bwd Packet Length Std', 'Flow Bytes/s', ' Flow Packets/s',
       ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min',
       'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max',
       ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean', ' Bwd IAT Std',
       ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags', ' Bwd PSH Flags',
       ' Fwd URG Flags', ' Bwd URG Flags', ' Fwd Header Length',
       ' Bwd Header Length', 'Fwd Packets/s', ' Bwd Packets/s',
       ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean',
       ' Packet Length Std', ' Packet Length Variance', '

In [ ]:
features = df.columns.tolist()
features.remove(' Label')
X = df[features]
y=df[' Label']


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)


In [ ]:
X_train.shape

(2264594, 78)

In [ ]:
print(X_train.dtypes)  # This will show the data type of each column
float_columns = X_train.select_dtypes(include=['float64']).columns


 Destination Port                 int64
 Flow Duration                    int64
 Total Fwd Packets                int64
 Total Backward Packets           int64
Total Length of Fwd Packets       int64
 Total Length of Bwd Packets      int64
 Fwd Packet Length Max            int64
 Fwd Packet Length Min            int64
 Fwd Packet Length Mean         float64
 Fwd Packet Length Std          float64
Bwd Packet Length Max             int64
 Bwd Packet Length Min            int64
 Bwd Packet Length Mean         float64
 Bwd Packet Length Std          float64
Flow Bytes/s                    float64
 Flow Packets/s                 float64
 Flow IAT Mean                  float64
 Flow IAT Std                   float64
 Flow IAT Max                     int64
 Flow IAT Min                     int64
Fwd IAT Total                     int64
 Fwd IAT Mean                   float64
 Fwd IAT Std                    float64
 Fwd IAT Max                      int64
 Fwd IAT Min                      int64


In [ ]:
import numpy as np

# Checking and replacing 'inf' with NaN
X_train_nan = np.where(np.isinf(X_train), np.nan, X_train)
X_test_nan = np.where(np.isinf(X_test), np.nan, X_test)

# Handling NaN values by imputing with the mean or median of the column
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='mean')  # or strategy='median'

X_train_imputed = imputer.fit_transform(X_train_nan)
X_test_imputed = imputer.transform(X_test_nan)

X_train_imputed = pd.DataFrame(X_train_imputed, columns=X_train.columns)
X_test_imputed = pd.DataFrame(X_test_imputed, columns=X_test.columns)

X_train_imputed[float_columns] = X_train_imputed[float_columns].round().astype('int64')
X_test_imputed[float_columns] = X_test_imputed[float_columns].round().astype('int64')

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_imputed)
X_test_scaled = scaler.transform(X_test_imputed)

In [ ]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
y_train_encoded = encoder.fit_transform(y_train)
y_test_encoded = encoder.transform(y_test)

In [ ]:
num_features = X_train_scaled.shape
num_classes = len(np.unique(y_train_encoded))

In [ ]:
from tensorflow.keras.layers import Dense


base_model.pop()  # Removes the last layer
base_model.add(Dense(num_classes, activation='softmax'))  # Add a new output layer

In [ ]:
from tensorflow.keras.models import Sequential
new_model = Sequential()
new_model.add(Dense(256, activation='relu', input_shape=(X_train_scaled.shape[1],)))


# Add all other layers from the original model except the input
for layer in base_model.layers[1:]:
    new_model.add(layer)


In [ ]:
for layer in new_model.layers[:-1]:  # Freeze all but the new output layer
    layer.trainable = False

In [ ]:
from tensorflow.keras.optimizers import Adam

# Recompile the model
new_model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Assuming you have your new dataset loaded and prepared in X_train, y_train, etc.
# Train the model (only the unfrozen layers will be updated)
history = new_model.fit(X_train_scaled, y_train_encoded, epochs=50, batch_size=500, validation_split=0.2)
test_loss, test_acc = new_model.evaluate(X_test_scaled, y_test_encoded)
print(f"Test Accuracy: {test_acc}")

Epoch 1/10
56615/56615 [==============================] - 157s 3ms/step - loss: 0.7502 - accuracy: 0.8027 - val_loss: 0.6885 - val_accuracy: 0.8032
Epoch 2/10
56615/56615 [==============================] - 153s 3ms/step - loss: 0.7163 - accuracy: 0.8044 - val_loss: 0.7005 - val_accuracy: 0.7985
Epoch 3/10
56615/56615 [==============================] - 151s 3ms/step - loss: 0.7169 - accuracy: 0.8044 - val_loss: 0.6887 - val_accuracy: 0.8007
Epoch 4/10
56615/56615 [==============================] - 151s 3ms/step - loss: 0.7193 - accuracy: 0.8046 - val_loss: 0.6915 - val_accuracy: 0.8054
Epoch 5/10
56615/56615 [==============================] - 151s 3ms/step - loss: 0.7179 - accuracy: 0.8045 - val_loss: 0.7006 - val_accuracy: 0.8034
Epoch 6/10
56615/56615 [==============================] - 152s 3ms/step - loss: 0.7174 - accuracy: 0.8045 - val_loss: 0.6917 - val_accuracy: 0.8039
Epoch 7/10
56615/56615 [==============================] - 153s 3ms/step - loss: 0.7158 - accuracy: 0.8045 - val_

In [ ]:
!kaggle datasets download -d azalhowaide/iot-dataset-for-intrusion-detection-systems-ids

100% 523M/525M [00:15<00:00, 38.1MB/s]
100% 525M/525M [00:15<00:00, 35.9MB/s]


In [ ]:
%%capture
!mv iot-dataset-for-intrusion-detection-systems-ids.zip /content/drive/MyDrive/
import pandas as pd
!unzip /content/drive/MyDrive/iot-dataset-for-intrusion-detection-systems-ids.zip

In [ ]:
iot = pd.read_csv('BoTNeTIoT-L01-v2.csv')

In [ ]:
iot.columns

Index(['MI_dir_L0.1_weight', 'MI_dir_L0.1_mean', 'MI_dir_L0.1_variance',
       'H_L0.1_weight', 'H_L0.1_mean', 'H_L0.1_variance', 'HH_L0.1_weight',
       'HH_L0.1_mean', 'HH_L0.1_std', 'HH_L0.1_magnitude', 'HH_L0.1_radius',
       'HH_L0.1_covariance', 'HH_L0.1_pcc', 'HH_jit_L0.1_weight',
       'HH_jit_L0.1_mean', 'HH_jit_L0.1_variance', 'HpHp_L0.1_weight',
       'HpHp_L0.1_mean', 'HpHp_L0.1_std', 'HpHp_L0.1_magnitude',
       'HpHp_L0.1_radius', 'HpHp_L0.1_covariance', 'HpHp_L0.1_pcc',
       'Device_Name', 'Attack', 'Attack_subType', 'label'],
      dtype='object')

In [ ]:
features = iot.columns.tolist()
features.remove('label')
features.remove('Attack')
features.remove('Attack_subType')
features.remove('Device_Name')
X = iot[features]
y=iot['Attack']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)


In [ ]:
float_columns = X_train.select_dtypes(include=['float64']).columns
# Checking and replacing 'inf' with NaN
X_train_nan = np.where(np.isinf(X_train), np.nan, X_train)
X_test_nan = np.where(np.isinf(X_test), np.nan, X_test)

# Handling NaN values by imputing with the mean or median of the column
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='mean')  # or strategy='median'

X_train_imputed = imputer.fit_transform(X_train_nan)
X_test_imputed = imputer.transform(X_test_nan)

X_train_imputed = pd.DataFrame(X_train_imputed, columns=X_train.columns)
X_test_imputed = pd.DataFrame(X_test_imputed, columns=X_test.columns)

X_train_imputed[float_columns] = X_train_imputed[float_columns].round().astype('int64')
X_test_imputed[float_columns] = X_test_imputed[float_columns].round().astype('int64')


In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_imputed)
X_test_scaled = scaler.transform(X_test_imputed)

In [ ]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
y_train_encoded = encoder.fit_transform(y_train)
y_test_encoded = encoder.transform(y_test)

In [ ]:
num_features = X_train_scaled.shape
num_classes = len(np.unique(y_train_encoded))

In [ ]:
from tensorflow.keras.layers import Dense


base_model.pop()  # Removes the last layer
base_model.add(Dense(num_classes, activation='softmax'))  # Add a new output layer

from tensorflow.keras.models import Sequential
new_model = Sequential()
new_model.add(Dense(256, activation='relu', input_shape=(X_train_scaled.shape[1],)))


# Add all other layers from the original model except the input
for layer in base_model.layers[1:]:
    new_model.add(layer)


In [ ]:
for layer in new_model.layers[:-1]:  # Freeze all but the new output layer
    layer.trainable = False

In [ ]:
from tensorflow.keras.optimizers import Adam

# Recompile the model
new_model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Assuming you have your new dataset loaded and prepared in X_train, y_train, etc.
# Train the model (only the unfrozen layers will be updated)
history = new_model.fit(X_train_scaled, y_train_encoded, epochs=10, batch_size=500, validation_split=0.2)
test_loss, test_acc = new_model.evaluate(X_test_scaled, y_test_encoded)
print(f"Test Accuracy: {test_acc}")

Epoch 1/10
9041/9041 [==============================] - 27s 3ms/step - loss: 0.7501 - accuracy: 0.6664 - val_loss: 0.6998 - val_accuracy: 0.8758
Epoch 2/10
9041/9041 [==============================] - 28s 3ms/step - loss: 0.7488 - accuracy: 0.6667 - val_loss: 0.6972 - val_accuracy: 0.8825
Epoch 3/10
9041/9041 [==============================] - 26s 3ms/step - loss: 0.7486 - accuracy: 0.6667 - val_loss: 0.6981 - val_accuracy: 0.8754
Epoch 4/10
9041/9041 [==============================] - 26s 3ms/step - loss: 0.7484 - accuracy: 0.6667 - val_loss: 0.6973 - val_accuracy: 0.8802
Epoch 5/10
9041/9041 [==============================] - 26s 3ms/step - loss: 0.7486 - accuracy: 0.6665 - val_loss: 0.6960 - val_accuracy: 0.8805
Epoch 6/10
9041/9041 [==============================] - 27s 3ms/step - loss: 0.7484 - accuracy: 0.6668 - val_loss: 0.6974 - val_accuracy: 0.8830
Epoch 7/10
9041/9041 [==============================] - 26s 3ms/step - loss: 0.7482 - accuracy: 0.6669 - val_loss: 0.6974 - val_ac